# Word Embeddings : le modèle Word2Vec

## Imports

In [65]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [66]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [67]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [68]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [69]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [70]:
len(bigram_phrases.vocab.keys())

15761694

Prenons une clé au hasard :

In [71]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

;'%


Le dictionnaire indique le score de cette coocurrence :

In [72]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [73]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [74]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [75]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [76]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [77]:
print(corpus[:100])

[['raege', 'quotidien', 'j00', 'exemplaires_distribues', 'gratuitement', '#', 'k', '.', "'", 'dans', 'ilagg', ';', 'lomeration', 'bruxelloise', 'et', '.', 'en_province', '.'], ['i', 'personnes_habitant', 'aux_etages', 'fenile', 'journal_contre', '60', 'c', '"<<', 'par_mois', '.'], ['fspir', 'est', 'expedie', 'en', 'proyinoe', 'contre', 't', '*>>"', 'par_mois', ',', 'payables_par_trimestre', '.'], ['annonces', ',', 'faits_divers', 'et', 'reclames', 'sjont', 'recu', 'frie', 'd', "'", 'isabelle', ';', 'a', 'l', "'", 'agence', 'rossfjl', '.'], ['44', '.'], ['-', 'v', ',', 'et', 'do', 'l', "'", 'enseignement', ',', 'a', 'braxell', 'annonces_remises_avant', 'midi', 'sont', ';', '-', 'a', ";'%", '>', '1', 'v', '>>^.'], [':', 'vv', '*', "*'*", 'hv', '/', 'f', '#-', 'v', '1', 'v', ';', "'", 'y', '>', 'fif', "'", 'ji', 'l', "'", 'h', '-', 'et', '.'], ['jeudi', '22', 'decembre', 'i887', 'observatoire', 'roy', "&'", 'cs', 'io', 'decembre', 's', 'heurts', 'uat', '$', 'r', '-:', 'r', 'quotidien_grat

## Entrainement d'un modèle Word2Vec sur ce corpus

In [78]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 4min 39s, sys: 14.5 s, total: 4min 53s
Wall time: 2min 1s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [79]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [80]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [81]:
model.wv["ministre"]

array([ 2.22267628e+00, -1.44581759e+00,  3.11505318e+00,  3.57834721e+00,
        5.26942074e-01,  2.63067341e+00,  5.02332354e+00, -3.05495113e-01,
       -1.09364426e+00, -3.56948876e+00, -3.95444822e+00,  3.75985813e+00,
        8.66313457e+00,  5.30918169e+00, -1.91254103e+00, -1.86902559e+00,
       -2.51020074e+00,  1.81631339e+00, -2.71324205e+00, -3.93816686e+00,
       -4.95131111e+00,  1.05519905e+01, -1.11366129e+00,  3.71533394e+00,
       -1.55611706e+00,  9.82813165e-03, -8.00381839e-01,  1.50562334e+00,
       -7.89586067e-01, -1.93595123e+00,  2.03643513e+00,  5.64186871e-01],
      dtype=float32)

### Calculer la similarité entre deux termes

In [82]:
model.wv.similarity("royalisme", "gentilhomme")

0.58848846

In [83]:
model.wv.similarity("attractions", "musique")

0.6452426

In [84]:
model.wv.similarity("geographie", "politique")

0.72318614

### Chercher les mots les plus proches d'un terme donné

In [85]:
model.wv.most_similar("royalisme", topn=10)

[('mien', 0.8289406299591064),
 ('caricaturiste', 0.8087123036384583),
 ('canari', 0.8037656545639038),
 ('herou_hotel', 0.7968170642852783),
 ('vrai_specialiste', 0.7907007336616516),
 ('petit_ange', 0.7856374979019165),
 ('plus_elegant', 0.7855172157287598),
 ('prunier', 0.7853228449821472),
 ('alien', 0.7848870158195496),
 ('monocle', 0.7818834185600281)]

In [86]:
model.wv.most_similar("attractions", topn=10)

[('ballets', 0.8834022879600525),
 ('danses', 0.8734057545661926),
 ('concerts', 0.8545930981636047),
 ('orchestres', 0.846609890460968),
 ('marionnettes', 0.8453210592269897),
 ('vedettes', 0.8322513103485107),
 ('bals', 0.8233698606491089),
 ('fanfares', 0.8210591673851013),
 ('choeurs', 0.8189030885696411),
 ('sonates', 0.814495861530304)]

In [87]:
model.wv.most_similar("geographie", topn=10)

[('medecine', 0.9106082320213318),
 ('pedagogie', 0.8973264694213867),
 ('litterature', 0.8965217471122742),
 ('sociologie', 0.8853312134742737),
 ('langue_francaise', 0.870191216468811),
 ('culturelle', 0.860663890838623),
 ('presse_belge', 0.8546460270881653),
 ('physiologie', 0.8473893404006958),
 ('philosophie', 0.845599889755249),
 ('vulgarisation', 0.8453320264816284)]

In [88]:
model.wv.most_similar("gentilhomme", topn=10)

[('brave_homme', 0.922004759311676),
 ('millionnaire', 0.9171417951583862),
 ('philosophe', 0.9138683080673218),
 ('pretre', 0.907555878162384),
 ('contentement', 0.9019019603729248),
 ('poete', 0.9018382430076599),
 ('compagnon', 0.9017373919487),
 ('romancier', 0.9009806513786316),
 ('galant', 0.8989869356155396),
 ('personnage', 0.8980175852775574)]

In [89]:
model.wv.most_similar("musique", topn=10)

[('danse', 0.8824560642242432),
 ('causerie', 0.8449584245681763),
 ('schubert', 0.8406160473823547),
 ('symphonie', 0.8381774425506592),
 ('mozart', 0.8377379775047302),
 ('beethoven', 0.8362671136856079),
 ('guitare', 0.8177639245986938),
 ('chopin', 0.8061898350715637),
 ('orchestre', 0.8055031895637512),
 ('verdi', 0.8026129007339478)]

In [90]:
model.wv.most_similar("politique", topn=10)

[('neutralite', 0.9278957843780518),
 ('la_democratie', 0.9077697396278381),
 ('democratique', 0.9002721905708313),
 ('doctrine', 0.8978691697120667),
 ('cooperation', 0.8972781896591187),
 ('solidarite', 0.8969495296478271),
 ('discipline', 0.8904068470001221),
 ('politique_exterieure', 0.8877522945404053),
 ('politique_interieure', 0.8842083215713501),
 ('politique_etrangere', 0.8822564482688904)]

### Faire des recherches complexes à travers l'espace vectoriel

In [91]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('berlin', 0.8138653635978699), ('datee_du', 0.7868233919143677), ('new_york', 0.7772507667541504), ('tokio', 0.7699856758117676), ('rome', 0.7483415007591248), ('moscou', 0.7378954291343689), ('casablanca', 0.7340309023857117), ('shanghai', 0.7336855530738831), ('teheran', 0.7257575392723083), ('par_avion', 0.7112868428230286)]
